In [ ]:
""" From "COMPUTATIONAL PHYSICS", 3rd Ed, Enlarged Python eTextBook  
    by RH Landau, MJ Paez, and CC Bordeianu
    Copyright Wiley-VCH Verlag GmbH & Co. KGaA, Berlin;  Copyright R Landau,
    Oregon State Unv, MJ Paez, Univ Antioquia, C Bordeianu, Univ Bucharest, 2015.
    Support by National Science Foundation"""                             
 
# Beam.py: solves Navier-Stokes equation for flow around beam
 
import matplotlib.pylab as p;
from mpl_toolkits.mplot3d import Axes3D;
from numpy import *;

print("Working, wait for the figure after 100 iterations")

Nxmax = 210;   Nymax = 60;   IL = 30;   H = 24;   T = 24;   h = 0.3 
u = zeros((Nxmax+1, Nymax+1), float)                    # Stream
w = zeros((Nxmax+1, Nymax+1), float)                    # Vorticity
V0 = 1.0;    omega = 0.1;   nu = 1.;   iter = 0;  R = V0 * h/nu    

def borders():                                          
    for i in range(0, Nxmax+1):                         # Init stream
       for j in range(0, Nymax+1):                      # Init vorticity
            w[i, j] = 0.
            u[i, j] = j * V0
    for i in range(0, Nxmax+1 ):                        # Fluid surface
      u[i, Nymax] = u[i, Nymax-1] + V0*h
      w[i, Nymax-1] = 0.  
    for j in range(0, Nymax+1):
        u[1, j] = u[0, j]
        w[0, j] = 0.                                    # Inlet
    for  i in range(0, Nxmax+1):                        # Centerline
       if i <=  IL and i>= IL+T:
           u[i, 0] = 0.
           w[i, 0] = 0. 
    for j in range(1, Nymax ):                          # Outlet
        w[Nxmax, j] = w[Nxmax-1, j] 
        u[Nxmax, j] = u[Nxmax-1, j]                   

def beam():                                              # BC for beam
   for  j in range (0, H+1):                             # Sides
      w[IL, j] =  - 2 * u[IL-1, j]/(h*h)                 # Front
      w[IL+T, j] =  - 2 * u[IL  +  T  +  1, j]/(h*h)     # Back
   for  i in range(IL, IL+T + 1): w[i, H - 1] =  - 2 * u[i, H]/(h*h);  
   for i in range(IL, IL+T+1):
     for j in range(0, H+1):
         u[IL, j] = 0.                                  # Front
         u[IL+T, j] = 0.                                # Back
         u[i, H] = 0;                                   # Top
         
def relax():                                            # Relax stream
    beam()                                              # Reset conditions  
    for i in range(1, Nxmax):                           # Relax stream
      for  j in range (1, Nymax):
        r1 = omega*((u[i+1,j]+u[i-1,j]+u[i,j+1]+u[i,j-1] + h*h*w[i,j])/4-u[i,j]) # please finish this line of SOR
        u[i, j] +=  r1   
    maxr2 = 0.0
    for  i in range(1, Nxmax):                          # Relax vorticity
      for j in range(1, Nymax):
          a1 = w[i+1, j]  +  w[i-1,j]  +  w[i,j+1] + w[i,j-1]
          a2 = (u[i,j+1] - u[i,j-1])*(w[i+1,j] - w[i - 1, j])
          a3 = (u[i+1,j] - u[i-1,j])*(w[i,j+1] - w[i, j - 1])
          r2 = omega *( (a1 - (R/4.)*(a2 - a3) )/4. - w[i,j])
          w[i, j] +=  r2
          if(r2 > maxr2):
            maxr2=r2
    print(maxr2)
            
borders()
while (iter <=  5000):
    iter +=  1
    if iter%10 ==  0: print (iter)
    relax()
for i in range (0, Nxmax+1):
    for  j in range(0, Nymax+ 1):  u[i,j] = u[i,j]/V0/h    # V0h units   
x = range(0, Nxmax-1);        y = range(0, Nymax-1)                                                        
X, Y = p.meshgrid(x, y)                                  

def functz(u):                                          # Stream flow
    z = u[X, Y]                                      
    return z

Z = functz(u)                                           
fig = p.figure()                                                 
ax = Axes3D(fig)                                   
ax.plot_wireframe(X, Y, Z, color = 'r')          
ax.set_xlabel('X')                                              
ax.set_ylabel('Y')
ax.set_zlabel('Stream Function')

#w.resize((70,70));
x=list(range(0,Nxmax-1))                   #to plot lines in x axis
y=list(range(0,Nymax-1))
#x=range(0,69)                   #to plot lines in x axis
#y=range(0,69)
X,Y=p.meshgrid(x,y)                  #grid for position and time

def functz(u):                       #returns stream flow to plot
    z = u[X,Y]                         #for several iterations
    return z

def functz1(w):                       #returns stream flow to plot
    z1 = w[X,Y]                         #for several iterations
    return z1
    
def functz2(u):
    z2=zeros((Nymax-1, Nxmax-1), float)   
    for i in range(0, Nxmax-1):                           # Relax stream
      for  j in range (0, Nymax-1):
        velx=(u[i+1,j]-u[i,j])/h
        vely=(u[i,j+1]-u[i,j])/h
        z2[j,i]=(velx*velx+vely*vely)**0.5
    return z2
    
Z = functz(u)
Z1 = functz1(w)
Z2 = functz2(u)

fig1 = p.figure()
p.title('Stream function - 2D Flow over a beam')
p.imshow(Z, origin='lower');
p.colorbar();

fig2=p.figure()
p.title('Vorticity - 2D Flow over a beam')
p.imshow(Z1, origin='lower');
p.colorbar();

fig3=p.figure()
p.title('Velocity - 2D Flow over a beam')
p.imshow(Z2, origin='lower');
p.colorbar();



p.show()                             # Shows the figure, close Python shell to
            

p.show()

Working, wait for the figure after 100 iterations
60.2825453072
49.7735719785
48.1590587969
46.8174349275
45.5395792358
44.3207923647
43.1567962161
42.0437204971
40.9780512395
10
39.9565894039
38.9764172608
38.034870629
37.1295154403
36.258127462
35.4186743243
34.6092992575
33.8283061387
33.0741455956
32.3454020139
20
31.6407813652
30.9590998117
30.2992730681
29.6603065137
29.0412860455
28.4413696649
27.8597797815
27.2957962152
26.7487498671
26.2180170283
30
25.7030142896
25.2031940117
24.7180403149
24.2470655459
23.7898071781
23.3458251064
22.9146992941
22.4960277354
22.0894246982
21.6945192132
40
21.3109537795
20.9383832591
20.5764739341
20.2249027059
19.8833564143
19.5515312594
19.2291323112
18.9158730914
18.6114752171
18.3156680946
50
18.0281886551
17.748781124
17.4771968183
17.2131939647
16.9565375364
16.7069991018
16.4643566845
16.22839463
15.998903479
15.7756798434
60
15.5585262856
15.3472511993
15.1416686906
14.9415984599
14.7468656838
14.5573008961
14.372739869
14.1930234935
1

3.27526137391
3.27219932602
3.26914656657
3.26610305028
3.26306873219
3.2600435676
3.25702751216
580
3.25402052176
3.25102255262
3.24803356123
3.24505350438
3.24208233912
3.2391200228
3.23616651306
3.23322176778
3.23028574516
3.22735840364
590
3.22443970194
3.22152959905
3.21862805423
3.21573502699
3.21285047711
3.20997436463
3.20710664985
3.20424729331
3.20139625583
3.19855349846
600
3.1957189825
3.19289266951
3.19007452128
3.18726449984
3.18446256749
3.18166868674
3.17888282034
3.1761049313
3.17333498283
3.17057293839
610
3.16781876167
3.16507241659
3.16233386728
3.15960307812
3.15688001368
3.15416463879
3.15145691846
3.14875681794
3.1460643027
3.14337933841
620
3.14070189095
3.13803192642
3.13536941114
3.13271431161
3.13006659456
3.12742622691
3.12479317578
3.1221674085
3.11954889261
3.11693759583
630
3.11433348607
3.11173653146
3.1091467003
3.1065639611
3.10398828254
3.10141963352
3.09885798309
3.09630330052
3.09375555524
3.09121471688
640
3.08868075523
3.0861536403
3.08363334223
3

2.31320644113
1150
2.3122704375
2.31133578881
2.31040249187
2.30947054346
2.3085399404
2.30761067951
2.30668275762
2.30575617157
2.30483091821
2.30390699441
1160
2.30298439704
2.30206312297
2.3011431691
2.30022453234
2.29930720959
2.29839119779
2.29747649385
2.29656309473
2.29565099737
2.29474019874
1170
2.29383069581
2.29292248556
2.29201556498
2.29110993107
2.29020558085
2.28930251133
2.28840071954
2.28750020253
2.28660095734
2.28570298102
1180
2.28480627066
2.28391082331
2.28301663608
2.28212370605
2.28123203033
2.28034160603
2.27945243028
2.2785645002
2.27767781295
2.27679236567
1190
2.27590815551
2.27502517966
2.27414343527
2.27326291955
2.27238362969
2.27150556288
2.27062871635
2.26975308731
2.268878673
2.26800547065
1200
2.26713347751
2.26626269083
2.26539310789
2.26452472595
2.2636575423
2.26279155422
2.26192675903
2.26106315401
2.26020073649
2.2593395038
1210
2.25847945326
2.25762058221
2.25676288801
2.25590636801
2.25505101958
2.25419684008
2.25334382691
2.25249197745
2.25164

1.93191413468
1720
1.9314314548
1.93094922996
1.93046745944
1.92998614255
1.92950527856
1.92902486677
1.92854490647
1.92806539696
1.92758633754
1.92710772751
1730
1.92662956615
1.92615185279
1.92567458671
1.92519776723
1.92472139364
1.92424546527
1.92376998141
1.92329494137
1.92282034448
1.92234619003
1740
1.92187247736
1.92139920576
1.92092637457
1.92045398309
1.91998203066
1.91951051659
1.9190394402
1.91856880083
1.91809859779
1.91762883042
1750
1.91715949805
1.9166906
1.91622213561
1.91575410421
1.91528650513
1.91481933772
1.91435260131
1.91388629523
1.91342041884
1.91295497147
1760
1.91248995247
1.91202536117
1.91156119694
1.9110974591
1.91063414702
1.91017126004
1.90970879752
1.9092467588
1.90878514325
1.90832395022
1770
1.90786317905
1.90740282913
1.90694289979
1.90648339041
1.90602430035
1.90556562897
1.90510737563
1.9046495397
1.90419212056
1.90373511756
1780
1.90327853008
1.90282235749
1.90236659917
1.90191125449
1.90145632281
1.90100180353
1.90054769602
1.90009399966
1.899640

1.71320903373
1.71290454245
1.71260026171
1.71229619125
1.71199233084
1.71168868023
1.71138523918
1.71108200744
1.71077898478
2300
1.71047617093
1.71017356568
1.70987116876
1.70956897995
1.70926699899
1.70896522566
1.7086636597
1.70836230088
1.70806114896
1.7077602037
2310
1.70745946486
1.7071589322
1.70685860548
1.70655848447
1.70625856893
1.70595885862
1.70565935331
1.70536005275
1.70506095671
1.70476206497
2320
1.70446337727
1.70416489339
1.7038666131
1.70356853615
1.70327066232
1.70297299137
1.70267552308
1.70237825719
1.70208119349
1.70178433175
2330
1.70148767173
1.70119121319
1.70089495592
1.70059889967
1.70030304423
1.70000738935
1.69971193482
1.6994166804
1.69912162586
1.69882677098
2340
1.69853211552
1.69823765927
1.69794340199
1.69764934345
1.69735548344
1.69706182172
1.69676835807
1.69647509226
1.69618202407
1.69588915328
2350
1.69559647966
1.69530400298
1.69501172302
1.69471963957
1.69442775239
1.69413606127
1.69384456598
1.6935532663
1.69326216202
1.6929712529
2360
1.6926

1.56723674135
1.56702216323
1.5668077017
1.56659335665
1.56637912797
1.56616501557
1.56595101933
2870
1.56573713915
1.56552337492
1.56530972654
1.56509619389
1.56488277688
1.5646694754
1.56445628934
1.5642432186
1.56403026307
1.56381742264
2880
1.56360469722
1.56339208669
1.56317959096
1.56296720991
1.56275494345
1.56254279147
1.56233075386
1.56211883052
1.56190702135
1.56169532624
2890
1.56148374509
1.5612722778
1.56106092426
1.56084968437
1.56063855802
1.56042754512
1.56021664556
1.56000585924
1.55979518605
1.5595846259
2900
1.55937417867
1.55916384428
1.55895362261
1.55874351356
1.55853351704
1.55832363294
1.55811386116
1.5579042016
1.55769465415
1.55748521872
2910
1.55727589521
1.55706668351
1.55685758352
1.55664859515
1.55643971829
1.55623095284
1.5560222987
1.55581375577
1.55560532396
1.55539700315
2920
1.55518879326
1.55498069418
1.55477270582
1.55456482807
1.55435706083
1.55414940401
1.5539418575
1.55373442122
1.55352709505
1.5533198789
2930
1.55311277268
1.55290577628
1.552698

1.4609012165
1.4607392966
1.46057744943
1.46041567493
1.46025397305
1.46009234373
3440
1.45993078692
1.45976930257
1.45960789061
1.45944655101
1.4592852837
1.45912408863
1.45896296574
1.45880191499
1.45864093632
1.45848002968
3450
1.45831919501
1.45815843225
1.45799774137
1.4578371223
1.45767657498
1.45751609938
1.45735569542
1.45719536307
1.45703510227
1.45687491296
3460
1.45671479509
1.45655474861
1.45639477347
1.45623486962
1.45607503699
1.45591527554
1.45575558522
1.45559596598
1.45543641775
1.4552769405
3470
1.45511753416
1.45495819869
1.45479893403
1.45463974013
1.45448061694
1.45432156441
1.45416258248
1.4540036711
1.45384483023
1.45368605981
3480
1.45352735978
1.4533687301
1.45321017071
1.45305168157
1.45289326262
1.45273491381
1.45257663509
1.4524184264
1.45226028771
1.45210221895
3490
1.45194422007
1.45178629102
1.45162843176
1.45147064223
1.45131292238
1.45115527215
1.45099769151
1.45084018039
1.45068273875
1.45052536654
3500
1.4503680637
1.45021083019
1.45005366596
1.449896

1.37857694031
1.37844935452
1.37832181804
4010
1.37819433085
1.37806689292
1.37793950422
1.37781216471
1.37768487436
1.37755763316
1.37743044105
1.37730329803
1.37717620404
1.37704915908
4020
1.37692216309
1.37679521607
1.37666831796
1.37654146875
1.37641466841
1.3762879169
1.37616121419
1.37603456026
1.37590795508
1.37578139861
4030
1.37565489082
1.37552843169
1.37540202118
1.37527565927
1.37514934593
1.37502308112
1.37489686482
1.374770697
1.37464457762
1.37451850666
4040
1.37439248409
1.37426650988
1.37414058399
1.37401470641
1.37388887709
1.37376309602
1.37363736316
1.37351167848
1.37338604195
1.37326045354
4050
1.37313491323
1.37300942099
1.37288397677
1.37275858057
1.37263323234
1.37250793206
1.3723826797
1.37225747523
1.37213231861
1.37200720983
4060
1.37188214885
1.37175713565
1.37163217019
1.37150725244
1.37138238238
1.37125755998
1.37113278521
1.37100805804
1.37088337843
1.37075874637
4070
1.37063416183
1.37050962477
1.37038513516
1.37026069298
1.3701362982
1.37001195079
1.36

1.31281824179
4580
1.31271464989
1.31261109367
1.3125075731
1.31240408818
1.31230063887
1.31219722517
1.31209384706
1.3119905045
1.3118871975
1.31178392603
4590
1.31168069006
1.31157748959
1.3114743246
1.31137119505
1.31126810095
1.31116504226
1.31106201898
1.31095903108
1.31085607854
1.31075316134
4600
1.31065027948
1.31054743292
1.31044462165
1.31034184566
1.31023910492
1.31013639942
1.31003372914
1.30993109405
1.30982849415
1.30972592941
4610
1.30962339982
1.30952090535
1.309418446
1.30931602173
1.30921363254
1.3091112784
1.3090089593
1.30890667522
1.30880442614
1.30870221205
4620
1.30860003292
1.30849788873
1.30839577948
1.30829370513
1.30819166568
1.30808966111
1.30798769139
1.30788575652
1.30778385646
1.30768199121
4630
1.30758016075
1.30747836505
1.3073766041
1.30727487789
1.30717318639
1.30707152959
1.30696990747
1.30686832001
1.3067667672
1.30666524901
4640
1.30656376543
1.30646231644
1.30636090202
1.30625952216
1.30615817684
1.30605686604
1.30595558974
1.30585434793
1.3057531